In [1]:
from mcp.server.fastmcp import FastMCP
from tavily import TavilyClient
from langchain_community.utilities import SearxSearchWrapper
import os
import httpx
from pathlib import Path

In [2]:
mcp = FastMCP("search api server")
print("✓ Created Web Search MCP server")

✓ Created Web Search MCP server


### Tool 1: Tavily search

In [3]:
@mcp.tool()
def search_tavily(query: str, max_results: int = 5) -> str:
    """
    Search the web using Tavily API.
    """
    api_key = os.getenv("TAVILY_API_KEY")
    print(api_key)
    if not api_key:
        raise ValueError("TAVILY_API_KEY environment variable is not set.")
    try:
        client = TavilyClient(api_key=api_key)
        results = client.search(query=query, max_results=max_results)
        formatted_res = []
        for i, res in enumerate(results.get("results",[]), 1):
            formatted_res.append(
                f"{i}. {res['title']}\n"
                f"   {res['url']}\n"
                f"   {res['content'][:200]}...\n"
            )
        return f"Tavily search results for '{query}':\n\n" + "\n".join(formatted_res)
    except Exception as e:
        return f"Tavily search error: {str(e)}"


### Tool 2: SearXNG search

In [4]:
@mcp.tool()
def search_searxng(query: str, max_results: int = 5) -> str:
    searxng_url = os.getenv("SEARXNG_URL")
    print(searxng_url)
    if not searxng_url:
        raise ValueError("SEARXNG_URL environment variable is not set.")
    try:
        searx = SearxSearchWrapper(searx_host=searxng_url)
        results = searx.results(query, num_results=max_results)
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('link', 'No URL')}\n"
                f"   {result.get('snippet', 'No snippet')[:150]}...\n"
            )
        
        return f"SearXNG search results for '{query}':\n\n" + "\n".join(formatted_results)
    except Exception as e:
        return f"SearXNG search error: {str(e)}"

In [5]:
# Resource: Search Configuration
@mcp.resource("config://search")
def get_search_config() -> str:
    """Get current search configuration"""
    config = {
        "providers": ["Tavily", "SearXNG"],
        "tavily_enabled": bool(os.getenv("TAVILY_API_KEY")),
        "searxng_url": os.getenv("SEARXNG_URL"),
        "default_max_results": 5
    }
    
    return f"Search Configuration:\n" + "\n".join([
        f"• {key}: {value}" for key, value in config.items()
    ])

print("✓ Added search config resource")

✓ Added search config resource


In [6]:
from dotenv import load_dotenv
load_dotenv("/home/siamai/deepsad/local-deep-researcher/.env")

True

In [7]:
# Test only the working parts
import os
print("Current env vars:")
print("TAVILY_API_KEY:", os.getenv("TAVILY_API_KEY"))
print("SEARXNG_URL:", os.getenv("SEARXNG_URL"))

# Test config only
config = get_search_config()
print(config)

Current env vars:
TAVILY_API_KEY: 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   # Get your key at https://tavily.com
SEARXNG_URL: http://localhost:8080
Search Configuration:
• providers: ['Tavily', 'SearXNG']
• tavily_enabled: True
• searxng_url: http://localhost:8080
• default_max_results: 5


In [14]:
def test_web_search():
    """Test web search functionality"""
    print("\n=== Testing Web Search Server ===")
    
    # # Test Tavily search
    # result = search_tavily("the illusion of reasoning 2025", 2)
    # print(f"Tavily result: {result[:200]}...")
    
    # Test SearXNG search
    result2 = search_searxng("the illusion of reasoning 2025", 5)
    print(f"SearXNG result: {result2[:2000]}...")
    
    # Test config
    config = get_search_config()
    print(f"Config: {config}")

print("✓ Web search server ready")

# Run test immediately
test_web_search()

✓ Web search server ready

=== Testing Web Search Server ===
http://localhost:8080
SearXNG result: SearXNG search results for 'the illusion of reasoning 2025':

1. The Illusion of Thinking: Understanding the Strengths and Limitations of Reasoning Models via the Lens of Problem Complexity - Apple Machine Learning Research
   https://machinelearning.apple.com/research/illusion-of-thinking
   However, the extensive reasoning traces lead to inefficiencies and an increased time-to-first-token (TTFT). We propose a novel training paradigm that ...

2. The illusion of "The Illusion of Thinking" | sean goedecke
   https://www.seangoedecke.com/illusion-of-thinking/
   The illusion of "The Illusion of Thinking" Very recently (early June 2025), Apple released a paper called The Illusion of Thinking: Understanding the ...

3. Intelligence Illusion: What Apple’s AI Study Reveals About Reasoning
   https://www.forbes.com/sites/corneliawalther/2025/06/09/intelligence-illusion-what-apples-ai-study-reve

In [12]:
import requests
import os

api_key = os.getenv("TAVILY_API_KEY")
url = "https://api.tavily.com/search"

headers = {
    "Content-Type": "application/json"
}

data = {
    "api_key": api_key,
    "query": "test",
    "max_results": 1
}

response = requests.post(url, json=data, headers=headers)
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

Status: 401
Response: {"detail":{"error":"Unauthorized: missing or invalid API key."}}
